## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
weather_data_df = pd.read_csv("../weather_database/WeatherPy_database.csv")
weather_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Ushuaia,AR,-54.8000,-68.3000,38.86,87,20,9.22,few clouds
1,1,Saint-Pierre,RE,-21.3393,55.4781,73.29,72,0,5.75,clear sky
2,2,Severo-Kurilsk,RU,50.6789,156.1250,36.23,85,83,8.43,broken clouds
3,3,Totma,RU,59.9747,42.7642,26.56,96,83,3.58,broken clouds
4,4,Copperas Cove,US,31.1241,-97.9031,80.80,61,75,11.50,broken clouds


In [8]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

# Ask the customer to add a minimum and maximum temperature value.
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [9]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

# 3. Filter the weather_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & \
                                       (weather_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Saint-Pierre,RE,-21.3393,55.4781,73.29,72,0,5.75,clear sky
4,4,Copperas Cove,US,31.1241,-97.9031,80.80,61,75,11.50,broken clouds
7,7,Vaini,TO,-21.2000,-175.2000,79.52,82,100,15.21,overcast clouds
8,8,Cabedelo,BR,-6.9811,-34.8339,77.67,83,23,9.71,few clouds
10,10,Rikitea,PF,-23.1203,-134.9692,76.59,70,100,6.20,overcast clouds
17,17,Half Moon Bay,US,37.4636,-122.4286,75.54,77,0,12.66,haze
19,19,Touros,BR,-5.1989,-35.4608,79.18,83,98,10.80,overcast clouds
21,21,Neiafu,TO,-18.6500,-173.9833,86.68,79,75,5.75,broken clouds
23,23,Bengkulu,ID,-3.8004,102.2655,78.94,81,99,3.85,overcast clouds
24,24,Cap Malheureux,MU,-19.9842,57.6142,73.47,80,2,7.76,clear sky


In [10]:
# 4a. Determine if there are any empty rows.

# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                272
City                   272
Country                271
Lat                    272
Lng                    272
Max Temp               272
Humidity               272
Cloudiness             272
Wind Speed             272
Current Description    272
dtype: int64

In [11]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
# 4b. Drop any empty rows and create a new DataFrame that doesn't have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                271
City                   271
Country                271
Lat                    271
Lng                    271
Max Temp               271
Humidity               271
Cloudiness             271
Wind Speed             271
Current Description    271
dtype: int64

In [12]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Saint-Pierre,RE,73.29,clear sky,-21.3393,55.4781,
4,Copperas Cove,US,80.80,broken clouds,31.1241,-97.9031,
7,Vaini,TO,79.52,overcast clouds,-21.2000,-175.2000,
8,Cabedelo,BR,77.67,few clouds,-6.9811,-34.8339,
10,Rikitea,PF,76.59,overcast clouds,-23.1203,-134.9692,
17,Half Moon Bay,US,75.54,haze,37.4636,-122.4286,
19,Touros,BR,79.18,overcast clouds,-5.1989,-35.4608,
21,Neiafu,TO,86.68,broken clouds,-18.6500,-173.9833,
23,Bengkulu,ID,78.94,overcast clouds,-3.8004,102.2655,
24,Cap Malheureux,MU,73.47,clear sky,-19.9842,57.6142,


In [13]:


# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [26]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.count()

City                   271
Country                271
Max Temp               271
Current Description    271
Lat                    271
Lng                    271
Hotel Name             271
dtype: int64

In [28]:
#8a. 
output_data_file = "../weather_database/Vacation_Search/weatherPy_vacation.csv"

hotel_df.to_csv(output_data_file, index_label="City_ID")

In [29]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Description</dt><dd>{Current Description}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]


# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [30]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]

max_temp = hotel_df["Max Temp"]

fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1)

marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))